In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dp2khegd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dp2khegd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=aedcdd2cc3422a2fb19ebe4f7df67ed897900c17d141df90d544bdf3c4017166
  Stored in directory: /tmp/pip-ephem-wheel-cache-qui6lyau/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <iostream>
using namespace std;
int main() {
    cout << "Hello";
    return 0;
}

Hello


In [ ]:
%%cu
#include <iostream>
#include <cstdlib>

#define N 4

// CUDA kernel for matrix multiplication
__global__ void matrixMul(int *a, int *b, int *c, int n)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < n && col < n) {
        int sum = 0;
        for (int i = 0; i < n; i++) {
            sum += a[row * n + i] * b[i * n + col];
        }
        c[row * n + col] = sum;
    }
}

int main()
{
    int *h_a, *h_b, *h_c; // host matrices
    int *d_a, *d_b, *d_c; // device matrices

    // Allocate host memory
    h_a = (int *)malloc(N * N * sizeof(int));
    h_b = (int *)malloc(N * N * sizeof(int));
    h_c = (int *)malloc(N * N * sizeof(int));

    // Allocate device memory
    cudaMalloc((void **)&d_a, N * N * sizeof(int));
    cudaMalloc((void **)&d_b, N * N * sizeof(int));
    cudaMalloc((void **)&d_c, N * N * sizeof(int));

    // Initialize host matrices
     for (int i = 0; i < N * N; i++) {
          h_a[i] = i + 1;
         h_b[i] = (i + 1) * 2;
     }
 
    // h_a[] = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16};
    // h_b[] = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16};

    // Copy host matrices to device
    cudaMemcpy(d_a, h_a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 blockDim(N, N);
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y);

    // Launch kernel
    matrixMul<<<gridDim, blockDim>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(h_c, d_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print result
    std::cout << "Matrix C:" << std::endl;
    for (int i = 0; i < N * N; i++) {
        std::cout << h_c[i] << " ";
        if ((i + 1) % N == 0) {
            std::cout << std::endl;
        }
    }

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}


Matrix C:
180 200 220 240 
404 456 508 560 
628 712 796 880 
852 968 1084 1200 



In [ ]:
%%cu
#include <iostream>
#include <cstdlib>

#define N 6

// CUDA kernel for vector addition
__global__ void vectorAdd(int *a, int *b, int *c, int n)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < n) {
        c[tid] = a[tid] + b[tid];
    }
}

int main()
{
    int *h_a, *h_b, *h_c; // host vectors
    int *d_a, *d_b, *d_c; // device vectors

    // Allocate host memory
    h_a = (int *)malloc(N * sizeof(int));
    h_b = (int *)malloc(N * sizeof(int));
    h_c = (int *)malloc(N * sizeof(int));

    // Allocate device memory
    cudaMalloc((void **)&d_a, N * sizeof(int));
    cudaMalloc((void **)&d_b, N * sizeof(int));
    cudaMalloc((void **)&d_c, N * sizeof(int));

    // Initialize host vectors
    for (int i = 0; i < N; i++) {
        h_a[i] = i;
        h_b[i] = 2 * i;
    }

    // Copy host vectors to device
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);

    // Define block and grid dimensions
    int blockSize = 256;
    int gridSize = (N + blockSize - 1) / blockSize;

    // Launch kernel
    vectorAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, N);

    // Copy result from device to host
    cudaMemcpy(h_c, d_c, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print result
    std::cout << "Vector C:" << std::endl;
    for (int i = 0; i < N; i++) {
        std::cout << h_c[i] << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Free host memory
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}


Vector C:
0 3 6 9 12 15 



In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <limits>
using namespace std;

#define N 15

// CUDA kernel for finding min, max, sum, and average
__global__ void arrayStats(int *arr, int size, int *minVal, int *maxVal, int *sum, float *average)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        // Initialize thread-local variables
        int localMin = INT_MAX;
        int localMax = INT_MIN;
        int localSum = 0;

        // Iterate over the array and update thread-local variables
        for (int i = tid; i < size; i += blockDim.x * gridDim.x) {
            int val = arr[i];
            localMin = min(localMin, val);
            localMax = max(localMax, val);
            localSum += val;
        }

        // Update global variables using atomic operations
        atomicMin(minVal, localMin);
        atomicMax(maxVal, localMax);
        atomicAdd(sum, localSum);
    }
}

int main()
{
    int *h_arr;       // host array
    int *d_arr;       // device array
    int *d_minVal;    // device minimum value
    int *d_maxVal;    // device maximum value
    int *d_sum;       // device sum
    float *d_average; // device average

    // Allocate host memory
    h_arr = (int *)malloc(N * sizeof(int));

    // Allocate device memory
    cudaMalloc((void **)&d_arr, N * sizeof(int));
    cudaMalloc((void **)&d_minVal, sizeof(int));
    cudaMalloc((void **)&d_maxVal, sizeof(int));
    cudaMalloc((void **)&d_sum, sizeof(int));
    cudaMalloc((void **)&d_average, sizeof(float));

    // Initialize host array
    for (int i = 0; i < N; i++) {
        h_arr[i] = rand() % 100;
    }

    // Copy host array to device
    cudaMemcpy(d_arr, h_arr, N * sizeof(int), cudaMemcpyHostToDevice);

    // Set initial values for min, max, sum, and average
    int initialMin = INT_MAX;
    int initialMax = INT_MIN;
    int initialSum = 0;
    float initialAverage = 0.0f;

    cudaMemcpy(d_minVal, &initialMin, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_maxVal, &initialMax, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_sum, &initialSum, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_average, &initialAverage, sizeof(float), cudaMemcpyHostToDevice);

    // Define block and grid dimensions
    int blockSize = 256;
    int gridSize = (N + blockSize - 1) / blockSize;

    // Launch kernel
    arrayStats<<<gridSize, blockSize>>>(d_arr, N, d_minVal, d_maxVal, d_sum, d_average);

    // Copy result from device to host
    int minVal, maxVal, sum;
    float average;
    cudaMemcpy(&minVal, d_minVal, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&maxVal, d_maxVal, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&sum, d_sum, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&average, d_average, sizeof(float), cudaMemcpyDeviceToHost);
 
    for(int i=0; i<N; i++) {
        cout << h_arr[i] << " ";
    }
    cout << "\n";

    // Print result
    std::cout << "Array Stats:" << std::endl;
    std::cout << "Min: " << minVal << std::endl;
    std::cout << "Max: " << maxVal << std::endl;
    std::cout << "Sum: " << sum << std::endl;
    std::cout << "Average: " << average << std::endl;

    // Free device memory
    cudaFree(d_arr);
    cudaFree(d_minVal);
    cudaFree(d_maxVal);
    cudaFree(d_sum);
    cudaFree(d_average);

    // Free host memory
    free(h_arr);

    return 0;
}


83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 
Array Stats:
Min: 15
Max: 93
Sum: 938
Average: 0

